<a href="https://colab.research.google.com/github/youseungH/changyi/blob/main/disastermessage_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요한 패키지

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta

openapi에서 다운 받고 BS4로 parsing 후 구조 파악

In [ ]:
# API endpoint 지정 및 원하는 정보 설정
api_url = 'http://apis.data.go.kr/1741000/DisasterMsg3/getDisasterMsg1List'
params = {
    'serviceKey': 'Nez6TJBxcgSSbgcW1kQj3ZU9WCc7I0S6Wu1PjkxUBz3u3iPwAIZZMddTf/Y16F//RXLPb//4EJ8Beh06JzlUFA==',
    'type': 'xml'
}


# API request
response = requests.get(api_url, params=params)

# 똑바로 됐는지 확인 및 text 출력해서 대략적인 구조 파악
'''
print(f"Status Code: {response.status_code}")
print("Response Headers:")
print(response.headers)
print("Response Text:")
print(response.text[:10000])  # Print the first 1000 characters for a quick glance
'''

# request 똑바로 안 됐을 때 실행됨.
if response.status_code != 200:
    print(f"Error: Unable to fetch data (status code: {response.status_code})")
    exit()

# BS4로 파싱
soup = BeautifulSoup(response.text, 'xml')

# 구조파악
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<DisasterMsg>
 <head>
  <totalCount>
   195578
  </totalCount>
  <numOfRows>
   100
  </numOfRows>
  <pageNo>
   1
  </pageNo>
  <type>
   XML
  </type>
  <RESULT>
   <resultCode>
    INFO-0
   </resultCode>
   <resultMsg>
    NOMAL SERVICE
   </resultMsg>
  </RESULT>
 </head>
 <row>
  <create_date>
   2024/07/23 20:12:50
  </create_date>
  <location_id>
   51
  </location_id>
  <location_name>
   경기도 하남시
  </location_name>
  <md101_sn>
   216175
  </md101_sn>
  <msg>
   하남시에서 배회 중인 방순월씨(여,74세)를 찾습니다-165cm,70kg,검정꽃무늬점퍼,검정바지,흰색마스크 vo.la/dsjaL / ☎112 [경기남부경찰청]
  </msg>
  <send_platform>
   cbs
  </send_platform>
 </row>
 <row>
  <create_date>
   2024/07/23 20:02:42
  </create_date>
  <location_id>
   30
  </location_id>
  <location_name>
   경기도 남양주시
  </location_name>
  <md101_sn>
   216174
  </md101_sn>
  <msg>
   남양주시에서 실종된 김숙희씨(여,60세)를 찾습니다-150cm,40kg,단발머리,회색긴팔줄무늬,검정크로스백 vo.la/BcRea / ☎112 [경기북부경찰청]
  </msg>
  <send_platform>
   cbs
  </send_plat

특정 날짜에 대한 Df 만들기

In [ ]:
# 날짜 선택 후, 그 날짜에 대해 Df 만들기
chosen_date='2024/07/21'

records=[]
for record in soup.find_all('row'):
    create_date=record.find('create_date').text.strip()
    if create_date.startswith(chosen_date):
        record_data={
            'create_date': create_date,
            'location_id': record.find('location_id').text.strip(),
            'location_name': record.find('location_name').text.strip(),
            'md101_sn': record.find('md101_sn').text.strip(),
            'msg': record.find('msg').text.strip(),
            'send_platform': record.find('send_platform').text.strip()
        }
        records.append(record_data)
df=pd.DataFrame(records)
print(df)

Empty DataFrame
Columns: []
Index: []


여러 날짜에 대해 Df 만들기

In [ ]:
# 특정 날짜를 입력받으면, 그 날짜에 대해 위에서 설정한 openapi에 있는 정보를 parsing한 후 각 정보를 dictionary로 저장해 하나의 list로 만들어 반환하는 함수
def fetch_date(chosen_date):
    response=requests.get(api_url, params=params)
    soup=BeautifulSoup(response.text, 'xml')

    records=[]
    for record in soup.find_all('row'):
        create_date = record.find('create_date').text.strip()
        if create_date.startswith(chosen_date):
            record_data = {
                'create_date': create_date,
                'location_id': record.find('location_id').text.strip(),
                'location_name': record.find('location_name').text.strip(),
                'md101_sn': record.find('md101_sn').text.strip(),
                'msg': record.find('msg').text.strip(),
                'send_platform': record.find('send_platform').text.strip()
            }
            records.append(record_data)
    return records

# 시작 날짜, 끝 날짜를 입력 받아 date_list를 만들기
start=input("시작 날짜를 입력하시오. (YYYY/MM/DD): ")
end=input("끝 날짜를 입력하시오. (YYYY/MM/DD): ")

start_date=datetime.strptime(start, '%Y/%m/%d')
end_date=datetime.strptime(end, '%Y/%m/%d')
date_list=[(start_date + timedelta(days=i)).strftime('%Y/%m/%d') for i in range((end_date - start_date).days + 1)]

# 만들어진 date_list에 대해 fetch_date 함수로 정보들을 불러와 하나의 Df로 만들기
df_list=[]
for date in date_list:
    records=fetch_date(date)
    df=pd.DataFrame(records)
    df_list.append(df)
df=pd.concat(df_list, ignore_index=True)

pd.set_option('display.max_rows', None) #모든 행 다 보기. 너무 길면 주석처리하고 보면 됨.

print(df)

시작 날짜를 입력하시오. (YYYY/MM/DD): 2024/07/20
끝 날짜를 입력하시오. (YYYY/MM/DD): 2024/07/24
            create_date                    location_id  \
0   2024/07/22 23:48:16                            136   
1   2024/07/22 23:32:43                             50   
2   2024/07/22 23:20:37                             14   
3   2024/07/22 22:32:11                             29   
4   2024/07/22 21:02:42                          42,48   
5   2024/07/22 21:00:08                             21   
6   2024/07/22 20:53:44                     21,136,168   
7   2024/07/22 20:38:50                             46   
8   2024/07/22 20:26:37                            231   
9   2024/07/22 19:47:50                             65   
10  2024/07/22 19:23:14                        127,128   
11  2024/07/22 19:13:17                        131,135   
12  2024/07/23 23:56:38              10353,10354,10355   
13  2024/07/23 20:12:50                             51   
14  2024/07/23 20:02:42                             3